In [9]:
kdprov = '76'

In [1]:
import pandas as pd

import geopandas as gpd
from glob import glob

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [10]:
# open data sp
data_peta_w_ruta = gpd.read_file("data/output/0_raster_data_desa/desa_pend_rasterize_"+kdprov+".gpkg").to_crs('ESRI:54009')#.area
data_peta_w_ruta['idkab']=data_peta_w_ruta.iddesa.apply(lambda y: str(y)[:4])


In [11]:
data_peta_w_ruta.head(4)

,iddesa,DN,id,pop2020,geometry,idkab
0,7601010001,0.0,0.0,5381.0,"MULTIPOLYGON (((11906900.000 -438300.000, 1190...",7601
1,7601010002,1.0,1.0,10635.0,"POLYGON ((11906700.000 -437200.000, 11907100.0...",7601
2,7601010003,2.0,2.0,5932.0,"POLYGON ((11908300.000 -435500.000, 11908600.0...",7601
3,7601010004,3.0,3.0,8330.0,"POLYGON ((11908300.000 -436000.000, 11908900.0...",7601


In [12]:
data_builtup_kabkota=glob("data/input/GHSL Data/2020/gpkg/ghs_built_"+kdprov+"*_nonzero.shp")

In [13]:
data_builtup_kabkota

['data/input/GHSL Data/2020/gpkg/ghs_built_7602_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_7605_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_7603_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_7606_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_7601_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_7604_nonzero.shp']

In [14]:
import os.path
def pop_dens(dt):
    global data_peta_w_ruta
    kdkab=dt.split('/')[5].replace('ghs_built_','').replace('_nonzero.shp','')
    if ((os.path.exists("data/output/1_population_density/grid_pop_"+kdkab+"_100m.gpkg")==False) 
        or (os.path.exists("data/output/1_population_density/grid_pop_"+kdkab+"_100m.gpkg-journal")==True)):
        data_builtup=gpd.read_file(dt)#.to_crs(('ESRI:54009'))
        data_builtup['DN']=data_builtup.value
        data_peta_w_ruta.pop2020.sum()
        data_builtup['centroid'] = data_builtup.centroid
        data_builtup['polygon'] = data_builtup['geometry']
        data_builtup.crs='ESRI:54009'
        data_peta_w_ruta.crs='ESRI:54009'
        data_builtup = data_builtup.loc[:,['DN','centroid','polygon']].set_geometry("centroid").sjoin(data_peta_w_ruta)
        data_builtup['TOTAL_PEND']=data_builtup['pop2020']
    # data_builtup = data_builtup.loc[:,['DN','geometry']].sjoin(data_peta_w_ruta)
        data_builtup['p_area'] = data_builtup['DN_left'] / data_builtup.groupby('iddesa').DN_left.transform('sum')
        data_builtup['j_pend'] = list(data_builtup['p_area']*data_builtup['TOTAL_PEND'])
    # data_builtup[['DN_left', 'geometry', 'iddesa','TOTAL_PEND', 'p_area','j_pend']].to_file("data/input/grid_pop_"+kdprov+"_100m.gpkg", driver="GPKG")
        data_builtup[['DN_left', 'centroid', 'iddesa','TOTAL_PEND', 'p_area','j_pend']].to_file("data/temp/grid_pop_"+kdkab+"_100m.gpkg", driver="GPKG")
        data_=data_builtup[['DN_left', 'polygon', 'iddesa','TOTAL_PEND', 'p_area','j_pend']].set_geometry("polygon")
        data_.crs='ESRI:54009'
        data_.to_file("data/output/1_population_density/grid_pop_"+kdkab+"_100m.gpkg", driver="GPKG")
    else:
        print('SKIP for: ',kdkab)

In [15]:
from tqdm.contrib.concurrent import process_map
process_map(pop_dens,data_builtup_kabkota)   

  0%|          | 0/6 [00:00<?, ?it/s]

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: ESRI:54009

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: ESRI:54009

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometr

[None, None, None, None, None, None]